In [ ]:

import json
with open('/home/sajadi/backup/tmp/wsd/detailedreslog.txt') as f:
    for line in f:
        jsline = line.split('\t')[1]
        js = json.loads(jsline)
        if ("wiki-mentions.5000.json" in js["dsname"]) and (js["ws"]==10):
            if js["method"]=='wlm':
                wlm = js["tp"]
            if js["method"]=='rvspagerank':
                rvs = js["tp"]
            if js["method"]=='ams':
                ams = js["tp"]                

In [ ]:
def get_prec(tp_list):
    overall_tp = 0
    overall_count=0
    macro_prec = 0;
    for tp, count in tp_list:
        overall_tp += tp
        overall_count += count
        macro_prec += float(tp)/count
        
    macro_prec = macro_prec/len(tp_list)
    micro_prec = float(overall_tp)/overall_count
    
    return micro_prec, macro_prec

In [ ]:
b=0
e=len(wlm)
e=5000

In [ ]:
print round(get_prec(wlm[b:e]))

In [ ]:
print get_prec(ams[b:e])

In [ ]:
print get_prec(rvs[b:e])

In [ ]:
round(.5)

# Some statistics for the paper

In [ ]:
l=0
i=0
with open('/home/sajadi/backup/tmpbackup/log.out.iterations.txt') as f:
    for line in f:
        r= re.match(".*# of iterations: ([\d]+),.*",line)
        if r is None:
            continue
        i += int(r.group(1))
        l +=1
print i,l, float(i)/l  


In [ ]:
(6.5+7.4)/2

# Processing results

In [ ]:
table_no=4
from utils import *

In [ ]:
import pandas as pd

df = pd.read_csv('/home/sajadi/backup/tmp/results/wsd/reslog.csv', sep='\t', header=0)
df.head()

In [ ]:
df1= df.drop('start', axis=1)
df1= df1.drop('ws', axis=1)
# df.drop(8, axis=1, inplace=True)

In [ ]:
df1.head()

In [ ]:
df2 = df1.copy()
df2.loc[:,'dsname'] = df2.loc[:,'dsname'].replace('/users/grad/sajadi/backup/datasets/ner/kore.json','KORE')
df2.loc[:,'dsname'] = df2.loc[:,'dsname'].replace('/users/grad/sajadi/backup/datasets/ner/aida.json','AIDA-CoNLL')
df2.loc[:,'dsname'] = df2.loc[:,'dsname'].replace('/users/grad/sajadi/backup/datasets/ner/wiki-mentions.5000.json','Wiki')



In [ ]:
def methodselect(r):
    if table_no==1:
        return (r.loc['method'] in ['wlm', 'ams', 'rvspagerank']) and (r.loc['dir']=='both') and \
                (r.loc['op_method'] == 'ilp')
    if table_no==2:
        return (r.loc['method'] in ['rvspagerank']) and (r.loc['dir']=='both') and \
               (r.loc['op_method'] in ['ilp', 'ilp2'])
    if table_no==3:
        return (r.loc['method'] == 'rvspagerank' and( r.loc['dir']=='both') and (r.loc['op_method'] in ['ilp','ilp2','context4_4']))
    if table_no==4:
        return (r.loc['method'] == 'rvspagerank') and (r.loc['dir']=='both') and (r.loc['op_method'] in ['ilp','context4_4']) 
                #(r.loc['method'] == 'wlm' and r.loc['dir']=='in' and r.loc['op_method'] in  ['tagme', 'tagme2'])
    
df3 = df2[df2.apply(lambda r: ('wiki-mentions.json' not in r.loc['dsname']) and (r.loc['max_t'] in [5,10,15]), axis=1)]
df3 = df3[df3.apply(methodselect, axis=1)]
df3

In [ ]:
df4=df3.drop('dir', axis=1)
if table_no==1:
    df4=df4.drop('op_method', axis=1)
if table_no==2 or table_no==3:
    df4=df4.drop('method', axis=1)
if table_no==4:
    df4=df4.drop(['method', 'micro', 'macro'], axis=1)
    
df4.head()

In [ ]:
cols = df4.columns.tolist()
cols

In [ ]:
cols = df4.columns.tolist()
cols[0],cols[1] = cols[1],cols[0]
df5 = df4[cols]
df5.head()

In [ ]:
if table_no==4:
    df5.loc[:,'time'] = df5.loc[:,'time'].apply(lambda x: str2delta(x).total_seconds())
df5    

In [ ]:
if table_no==1:
    grouped = df5.groupby(['max_t','method'])
if table_no in [2,3,4]:
    grouped = df5.groupby(['max_t','op_method'])
#grouped.get_group((5,'ams'))

In [ ]:
for g in grouped.groups:
    print grouped.get_group(g)
    break

In [ ]:
rdf = pd.DataFrame()
for g in grouped.groups:
    gdf = grouped.get_group(g)
    tkore = gdf[gdf['dsname'] == "KORE"]
    taida = gdf[gdf['dsname'] == "AIDA-CoNLL"]
    twiki = gdf[gdf['dsname'] == "Wiki"]
    if table_no==1:
        tdf = pd.DataFrame([[gdf['max_t'].iloc[0], gdf['method'].iloc[0]] 
                        + list(tkore.iloc[0,3:5])+ list(taida.iloc[0,3:5]) + list(twiki.iloc[0,3:5]) ])
    if table_no==2 or table_no==3:
        tdf = pd.DataFrame([[gdf['max_t'].iloc[0], gdf['op_method'].iloc[0]] 
                        + list(tkore.iloc[0,3:5])+ list(taida.iloc[0,3:5]) + list(twiki.iloc[0,3:5]) ])
    if table_no==4:
        tdf = pd.DataFrame([[gdf['max_t'].iloc[0], gdf['op_method'].iloc[0]] 
                        + list(tkore.iloc[0,3:])+ list(taida.iloc[0,3:]) + list(twiki.iloc[0,3:]) ])
    rdf = rdf.append(tdf)

In [ ]:
rdf

In [ ]:
#miindex = pd.MultiIndex.f(index, names=['cs', 'SR Method'])
midf = rdf.copy().sort_values(by=[0,1])
midf

In [ ]:
midf.index = pd.MultiIndex.from_arrays([midf[0], midf[1]],names=['cs', 'Method'])
midf

In [ ]:
midf.drop(0, axis=1, inplace=True)
midf.drop(1, axis=1, inplace=True)
midf

In [ ]:
if table_no in [1,2,3]:
    cols = [["KORE","AIDA-CoNLL","Wiki"],["Micro","Macro"]]
    micolumns = pd.MultiIndex.from_product(cols, names=['Dataset', 'Precision'])
if table_no in [4]:
    micolumns = ["KORE","AIDA-CoNLL","Wiki"]
    
midf.columns = micolumns
midf = midf.round(2)
midf

In [ ]:
print midf.to_latex()

In [ ]:
midf

In [ ]:
x=pd.Series()
y=pd.Series()
l=[]
for i in midf.index.levels[0]:
        x=x.append(midf.loc[(i,'ilp')])
        y=y.append(midf.loc[(i,'context4_4')])
        l += [c+'\n'+str(i) for c in midf.columns]
x=x.values
y=y.values
        

In [ ]:
l

In [ ]:
import numpy as np
import matplotlib.pyplot as plt


fig, ax = plt.subplots()

index = np.arange(len(x))
bar_width = 0.45

opacity = 0.4
error_config = {'ecolor': '0.3'}

rects1 = plt.bar(index, x, bar_width,
                 alpha=opacity,
                 color='b',
                 error_kw=error_config,
                 label='Coherence (ILP)')

rects2 = plt.bar(index + bar_width, y, bar_width,
                 alpha=opacity,
                 color='r',
                 error_kw=error_config,
                 label='Context Based Selective')

plt.xlabel('Datasets and Candidate Numbers')
plt.ylabel('Time (Seconds)')
plt.xticks(index + bar_width, l)
plt.legend()

plt.tight_layout()
plt.savefig('timebar.png')
plt.show()


In [ ]:
performance

In [ ]:
import plotly.plotly as py
import pandas as pd
from plotly.graph_objs import *

budget=pd.read_csv("mn-budget-detail-2014.csv")
budget.sort('amount',ascending=False,inplace=True)
budget = budget[:10]

# Semantic Resuls


In [ ]:
import pandas as pd
df = pd.read_csv('/home/sajadi/backup/projects/reports/semanticresults/outputs/spearman/spearman2.csv', sep=',', header=0)
df.head()

In [ ]:
df1= df.drop('wordsim_similarity-edited', axis=1)
df1= df1.drop('wordsim_relatedness-edited', axis=1)
df1= df1.drop('minimayo.1', axis=1)
df1= df1.drop('minimayo.3', axis=1)
#df1 = df1.columns.rename({'Unnamed: 0':'method'})
df1=df1.rename(columns={'Unnamed: 0':'method'})

In [ ]:
def methodselect(r):
    return r.loc['method'] in \
        ['in/no/results.txt','in/nged/results.txt', 'in/hits/pagerank/results.cos.txt',  'in/hits/rvspagerank/results.cos.txt',
         'out/no/results.txt','out/nged/results.txt', 'out/hits/pagerank/results.cos.txt',  'out/hits/rvspagerank/results.cos.txt',
         'all/no/results.txt','all/nged/results.txt', 'all/hits/pagerank/results.cos.txt',  'all/hits/rvspagerank/results.cos.txt',
         'hitsboth.avg.rvspagerank.rvspagerank/results.cos.txt',
        ]
    
df2 = df1[df1.apply(methodselect, axis=1)]
df2

In [ ]:
idx = df2.loc[:,'method']
idx=idx.apply(lambda x: x[0:3])
df3=df2.copy()
df3.insert(0,'dir',idx)
df3

In [ ]:
df4 = df3.copy().sort_values(by=['dir','method'])
df4.index = pd.MultiIndex.from_arrays([df4['dir'], df4['method']],names=['dir', 'Method'])
df4=df4.drop('dir', axis=1)
df4=df4.drop('method', axis=1)

df4

In [8]:
import sys
sys.path.insert(0, '..')
import wikisim.wikipedia